In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,596 kB]
Hit:12 http://ppa.launchpad.net/cra

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-25 18:41:39--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.22MB/s    in 0.2s    

2022-02-25 18:41:39 (5.22 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# starting Spark session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Import Data

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
mobile_apps_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"), sep = r"\t", header = True, inferSchema = True)

# Show DataFrame
mobile_apps_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

# Create and Clean New DataFrames for Customers, Products and Reviews

In [5]:
# Create reviews dataframe to match mobile_apps_df table
apps_reviews_df = mobile_apps_df.select(["review_id", "customer_id", "product_id",
                                         "product_parent", "review_date"])
apps_reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3PMQXEVYR4ZT1|    1774101|B00DVKOYBM|     263046246| 2015-08-31|
|R1I1K3EO98EMUV|   13679234|B00R1IA4RS|     128868854| 2015-08-31|
|R3K23URSI2MY42|   15056632|B01069LGHK|     955481764| 2015-08-31|
| RH2P770T10X38|   52477711|B00X64ZKM0|     455029830| 2015-08-31|
| RV9YBAW8CWFYN|   15021042|B00KIZO238|     685358159| 2015-08-31|
| R19627K7G1G6X|    5181462|B00H9FE0FU|     158085302| 2015-08-31|
|R22858XECEF7YQ|   19360265|B0064X7FVE|     140639925| 2015-08-31|
|R3KQOY7MDZOK0U|   28928572|B00N28818A|     838378762| 2015-08-31|
|R31BKIN02QFDHS|   26842555|B00HGMXSGI|     141073643| 2015-08-31|
|R3FW62P5EUDUUV|   25535583|B0119R2DB6|     498391460| 2015-08-31|
| RBD8ROQJQO68E|    3343825|B00O0OR65E|     156086007| 2015-08-31|
|R1S04R7IT12FKH|   49181739|B00YCL9R0W|     629058532| 2015-08

In [6]:
# checking the datatypes of each column
apps_reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# changing review_date to 'date' data type in "yyyy-mm-dd" format
apps_reviews_df = apps_reviews_df.withColumn("review_date",to_date(col("review_date"),"yyyy-mm-dd"))

In [8]:
# confirming change has been made
apps_reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
# counting number of rows in apps_reviews_df
apps_reviews_df.count()

5033376

In [10]:
# Create products dataframe to match mobile_apps_df table and removing duplicates
apps_products_df = mobile_apps_df.select(["product_id", "product_title"])
apps_products_df = apps_products_df.drop_duplicates()
apps_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00NG6CIKA|Solitaire TriPeak...|
|B00ZDVV7Q2|    Frozen Free Fall|
|B00N0RK9JM|How Draw Adventur...|
|B00SYZDSXO|       Bee Brilliant|
|B00HAPRVWS|Sonic The Hedgehog 2|
|B00UQK19Z4|HD Screen Wallpap...|
|B0073VRX5Y|Power-Button Flas...|
|B00ZHDCTLS|           bunny car|
|B00I3MX20W|            Vanguard|
|B00ITR6WLW|How to Draw: Eque...|
|B00CREXAVS|           Bridge V+|
|B00CSQYYHO|     Where's My Cat?|
|B009KVP70S|          War Sounds|
|B00KQ1XYFQ|   My Verizon Mobile|
|B014GD3NRS|          kumo cloud|
|B00EDUTGB2|     HP Print Plugin|
|B00NXOLPFY|Horse Simulator 3...|
|B004AHBBPW|         Daily Bible|
|B008I43DYU|      Calculator Pro|
|B00RG80MSY|        Jewels Blitz|
+----------+--------------------+
only showing top 20 rows



In [11]:
# checking the datatypes of each column
apps_products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [22]:
# counting number of rows in apps_products_df
apps_products_df.count()

127064

In [12]:
# creating customer dataframe and adding "customer count" as a column
apps_customers_df = apps_reviews_df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
apps_customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   44552694|             7|
|   27457155|             6|
|   35842367|             9|
|   16848166|             5|
|   13313689|            10|
|    3628854|             1|
|   43626894|             1|
|    4658983|             8|
|   16457323|             3|
|   30024726|             5|
|    3636174|             1|
|     582750|             1|
|      78113|             1|
|    2246607|             5|
|   26264517|             1|
|    7439900|             1|
|    2361059|             2|
|   22965400|             2|
|   32777038|             1|
|   29019526|             2|
+-----------+--------------+
only showing top 20 rows



In [23]:
# checking the datatypes of each column
apps_customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [24]:
# counting number of rows in apps_customers_df
apps_customers_df.count()

2356711

# Configuration for RDS Instance

In [17]:
# Configure settings for RDS

## IMPORTANT : PLEASE ADD YOUR OWN CREDENTIALS FOR RDS ##

mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"username", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

# Write Dataframes to RDS

In [19]:
# Write DataFrame to apps_customers table in RDS

apps_customers_df.write.jdbc(url=jdbc_url, table='apps_customers', mode=mode, properties=config)

In [20]:
# Write dataframe to apps_products table in RDS

apps_products_df.write.jdbc(url=jdbc_url, table='apps_products', mode=mode, properties=config)

In [21]:
# Write dataframe to apps_review_id_table in RDS

apps_reviews_df.write.jdbc(url=jdbc_url, table='apps_review_id_table', mode=mode, properties=config)